In [2]:
import polars as pl

# Load dino dataset
url_dino = 'https://raw.githubusercontent.com/kolibril13/ipydrop/refs/heads/main/dino.csv'
df_dino = pl.read_csv(url_dino)
dino_data = df_dino.with_columns(
    pl.col("x").round(4).alias("Dino_X"),
    pl.col("y").round(4).alias("Dino_Y")
).select(["Dino_X", "Dino_Y"])

# Load star dataset
url_star = 'https://raw.githubusercontent.com/kolibril13/ipydrop/refs/heads/main/star.csv'
df_star = pl.read_csv(url_star)
star_data = df_star.with_columns(
    pl.col("x").round(4).alias("Star_X"),
    pl.col("y").round(4).alias("Star_Y")
).select(["Star_X", "Star_Y"])

# Combine into one DataFrame
combined_df = pl.concat([dino_data, star_data], how="horizontal")

# Save to CSV
combined_df.write_csv("deno_star_elemens.csv")
print("File saved as 'deno_star_elemens.csv'")

File saved as 'deno_star_elemens.csv'


# parquet, with vector entries in cells

In [7]:
import polars as pl

url_dino = "https://raw.githubusercontent.com/kolibril13/ipydrop/refs/heads/main/dino.csv"
url_star = "https://raw.githubusercontent.com/kolibril13/ipydrop/refs/heads/main/star.csv"

(
    pl.concat([
        pl.read_csv(url_dino).select(
            pl.col("x").round(4).alias("dino_x"),
            pl.col("y").round(4).alias("dino_y")
        ),
        pl.read_csv(url_star).select(
            pl.col("x").round(4).alias("star_x"),
            pl.col("y").round(4).alias("star_y")
        ),
    ], how="horizontal")
    .with_columns(
        Dino = pl.concat_list(["dino_x", "dino_y"]),
        Star = pl.concat_list(["star_x", "star_y"])
    )
    .select(["Dino", "Star"])
    .write_parquet("dino_star_vectors.parquet")
)

print("File saved as 'dino_star_vectors.parquet'")

File saved as 'dino_star_vectors.parquet'


# json, but unwanted column orientation

In [5]:
import polars as pl

url_dino = "https://raw.githubusercontent.com/kolibril13/ipydrop/refs/heads/main/dino.csv"
url_star = "https://raw.githubusercontent.com/kolibril13/ipydrop/refs/heads/main/star.csv"

(
    pl.concat([
        pl.read_csv(url_dino).select(
            pl.col("x").round(4).alias("dino_x"),
            pl.col("y").round(4).alias("dino_y")
        ),
        pl.read_csv(url_star).select(
            pl.col("x").round(4).alias("star_x"),
            pl.col("y").round(4).alias("star_y")
        ),
    ], how="horizontal")
    .with_columns(
        Dino = pl.concat_list(["dino_x", "dino_y"]),
        Star = pl.concat_list(["star_x", "star_y"])
    )
    .select(["Dino", "Star"])
    .write_json("dino_star_vectors.json")
)

print("File saved as 'dino_star_vectors.json'")

File saved as 'dino_star_vectors.json'


# json, good column names

In [3]:
import polars as pl
import json

url_dino = "https://raw.githubusercontent.com/kolibril13/ipydrop/refs/heads/main/dino.csv"
url_star = "https://raw.githubusercontent.com/kolibril13/ipydrop/refs/heads/main/star.csv"

df = (
    pl.concat([
        pl.read_csv(url_dino).select(
            pl.col("x").round(4).alias("dino_x"),
            pl.col("y").round(4).alias("dino_y")
        ),
        pl.read_csv(url_star).select(
            pl.col("x").round(4).alias("star_x"),
            pl.col("y").round(4).alias("star_y")
        ),
    ], how="horizontal")
    .with_columns(
        Dino = pl.concat_list(["dino_x", "dino_y"]),
        Star = pl.concat_list(["star_x", "star_y"])
    )
    .select(["Dino", "Star"])
)

# Convert the DataFrame to a dictionary of lists
data_dict = df.to_dict(as_series=False)

# data_dict now looks like:
# {
#    "Dino": [[55.3846, 97.1795], [51.5385, 96.0256], ...],
#    "Star": [[58.2136, 91.8819], [58.1961, 92.215], ...]
# }

# Write the column-oriented data to JSON
with open("dino_star_vectors.json", "w") as f:
    json.dump(data_dict, f, indent=2)

print("File saved as 'dino_star_vectors.json'")

File saved as 'dino_star_vectors.json'


In [17]:
df = pl.read_json("/Users/jan-hendrik/projects/blender_csv_import/generate_data/dino_star_vectors.json")
# Explode the columns to transform list[list] into individual rows
df = df.explode(["Dino", "Star"])
df

Dino,Star
list[f64],list[f64]
"[55.3846, 97.1795]","[58.2136, 91.8819]"
"[51.5385, 96.0256]","[58.1961, 92.215]"
"[46.1538, 94.4872]","[58.7182, 90.3105]"
"[42.8205, 91.4103]","[57.2784, 89.9076]"
"[40.7692, 88.3333]","[58.082, 92.0081]"
…,…
"[39.4872, 25.3846]","[43.7226, 19.0773]"
"[91.2821, 41.5385]","[79.3261, 52.9004]"
"[50.0, 95.7692]","[56.664, 87.9401]"


In [2]:
import polars as pl
import json

url_dino = "https://raw.githubusercontent.com/kolibril13/ipydrop/refs/heads/main/dino.csv"
url_star = "https://raw.githubusercontent.com/kolibril13/ipydrop/refs/heads/main/star.csv"

df = (
    pl.concat([
        pl.read_csv(url_dino).select(
            pl.col("x").round(4).alias("dino_x"),
            pl.col("y").round(4).alias("dino_y")
        ),
        pl.read_csv(url_star).select(
            pl.col("x").round(4).alias("star_x"),
            pl.col("y").round(4).alias("star_y")
        ),
    ], how="horizontal")
    .with_columns(
        Dino = pl.concat_list(["dino_x", "dino_y", pl.lit(0).alias("dino_z")]),
        Star = pl.concat_list(["star_x", "star_y", pl.lit(0).alias("star_z")])
    )
    .select(["Dino", "Star"])
)

# Convert the DataFrame to a dictionary of lists
data_dict = df.to_dict(as_series=False)

# Write the column-oriented data to JSON
with open("dino_star_vectors_3d_vector.json", "w") as f:
    json.dump(data_dict, f, indent=2)

print("File saved as 'dino_star_vectors_3d_vector.json'")


File saved as 'dino_star_vectors_3d_vector.json'
